# User-Defined Types: Classes

In [39]:
import contextlib
from dataclasses import dataclass
from typing import List, Optional

## Class anatomy

In [3]:
# You can write a class just like a dataclass:

class Person:
    name: str = ""
    years_experience: int = 0
    address: str = ""

pat = Person()
pat.name = "Pat"
print(f"Hello {pat.name}")


Hello Pat


In [7]:
# The last class can be easily writen it a different way with a dict or dataclass

pat = {
    "name": "",
    "years_experience": 0,
    "address": ""
}

@dataclass
class Person():
    name: str = ""
    years_experience: int = 0
    address: str = ""


### Constructors

In [12]:
# You define a constructor with an __init__ method:

class Person:
    def __init__(
        self, 
        name: str, 
        years_experience: int,
        address: str
    ):
        self.name = name
        self.years_experience = years_experience
        self.address = address

pat = Person("Pat", 13, "123 Fake St.")


### Invariants

```
Consider the imaginary automated pizza maker again. We can encode the following invariants about it:
• Sauce will never be put on top of toppings (cheese is a topping in this scenario).
• Toppings may go above or below cheese.
• Pizza will have at most only one sauce.
• Dough radius can be only whole numbers.
• The radius of dough may be only between 6 and 12 inches, inclusive (between 15 and 30 centimeters).
```

In [14]:
# The last invariants can be encoded in the constructor, as follows

def is_sauce(t):
    return 

class PizzaSpecification:
    def __init__(
        self,
        dough_radius_in_inches: int,
        toppings: list[str]
    ):
        assert 6 <= dough_radius_in_inches <= 12, 'Dough must be between 6 and 12 inches'
        sauces = [t for t in toppings if is_sauce(t)]
        
        assert len(sauces) < 2, 'Can only have at most one sauce'
        self.dough_radius_in_inches = dough_radius_in_inches
        sauce = sauces[:1]
        self.toppings = sauce + [t for t in toppings if not is_sauce(t)]


In [17]:
# Alternatively, classes can be instantiated with a factory function, as follows

class _PizzaSpecification:
    # ... snip class
    pass

def create_pizza_spec(
    dough_radius_in_inches: int,
    toppings: list[str]
) -> Optional[_PizzaSpecification]:
    try:
        return _PizzaSpecification()
    except:
        return None


In [18]:
# Consider a pizza specification represented by a dictionary:
pizzas = {
    "dough_radius_in_inches": 7,
    "toppings": ["tomato sauce", "mozzarella", "pepperoni"],
}


In [22]:
# Invariants can be communicated in class docstrings, as follows

class PizzaSpecification:
    """
    This class represents a Pizza Specification for use in
    Automated Pizza Machines.
    The pizza specification is defined by the size of the dough and
    the toppings. Dough should be a whole number between 6 and 12
    inches (inclusive). If anything else is passed in, an AssertionError
    is thrown. The machinery cannot handle less than 6 inches and the
    business case is too costly for more than 12 inches.
    Toppings may have at most one sauce, but you may pass in toppings
    in any order. If there is more than one sauce, an AssertionError is
    thrown. This is done based on our research telling us that
    consumers find two-sauced pizzas do not taste good.
    This class will make sure that sauce is always the first topping,
    regardless of order passed in.
    Toppings are allowed to go above and below cheese
    (the order of non-sauce toppings matters).
    """
    def __init__(self):
        # ... implementation goes here
        pass


In [27]:
# Invariants can be tested with the help of context managers
# (to force code to run when a with block is exited), as follows

@contextlib.contextmanager
def create_pizza_specification(
    dough_radius_in_inches: int,
    toppings: list[str]
):
    pizza_spec = PizzaSpecification(dough_radius_in_inches, toppings)
    yield pizza_spec
    
    assert 6 <= pizza_spec.dough_radius_in_inches <= 12
    sauces = [t for t in pizza_spec.toppings if is_sauce(t)]
    
    assert len(sauces) < 2
    if sauces:
        assert pizza_spec.toppings[0] == sauces[0]
    
    # Check that we assert order of all non sauces.
    # Keep in mind, no invariant is specified that we can't add
    # toppings at a later date, so we only check against what was passed in.
    non_sauces = [t for t in pizza_spec.toppings if t not in sauces]
    expected_non_sauces = [t for t in toppings if t not in sauces]
    for expected, actual in zip(expected_non_sauces, non_sauces):
        assert expected == actual


def test_pizza_operations():
    with create_pizza_specification(
        8, ["Tomato Sauce", "Peppers"]
    ) as pizza_spec:
        # do something with pizza_spec
        pass


### Protecting Data Access


In [31]:
# Consider the class PizzaSpecification again:

def is_sauce(t):
    return 

class PizzaSpecification:
    def __init__(
        self,
        dough_radius_in_inches: int,
        toppings: list[str]
    ):
        assert 6 <= dough_radius_in_inches <= 12, 'Dough must be between 6 and 12 inches'
        sauces = [t for t in toppings if is_sauce(t)]
        
        assert len(sauces) < 2, 'Can only have at most one sauce'
        self.dough_radius_in_inches = dough_radius_in_inches
        sauce = sauces[:1]
        self.toppings = sauce + [t for t in toppings if not is_sauce(t)]

pizza_spec = PizzaSpecification(
    dough_radius_in_inches=8,
    toppings=['Olive Oil', 'Garlic', 'Sliced Roma Tomatoes', 'Mozzarella']
)


In [32]:
# Rigth now, nothing is preventing a future developer 
# from changing some invariants after the fact

pizza_spec.dough_radius_in_inches = 100
pizza_spec.dough_radius_in_inches


100

In [33]:
pizza_spec.toppings.append('Tomato Sauce')
pizza_spec.toppings

['Olive Oil',
 'Garlic',
 'Sliced Roma Tomatoes',
 'Mozzarella',
 'Tomato Sauce',
 'Tomato Sauce']

In [34]:
# Consider the PizzaSpecification class again, with private members:

class PizzaSpecification:
    def __init__(
        self,
        dough_radius_in_inches: int,
        toppings: list[str]
    ):
        assert 6 <= dough_radius_in_inches <= 12, 'Dough must be between 6 and 12 inches'
        sauces = [t for t in toppings if is_sauce(t)]
        
        assert len(sauces) < 2, 'Can have at most one sauce'
        self.__dough_radius_in_inches = dough_radius_in_inches
        sauce = sauces[:1]
        self.__toppings = sauce + [t for t in toppings if not is_sauce(t)]

pizza_spec = PizzaSpecification(
    dough_radius_in_inches=8,
    toppings=['Olive Oil', 'Garlic', 'Sliced Roma Tomatoes','Mozzarella']
)


In [36]:
# Now, trying to update a private attribute will raise an Exception

pizza_spec.__toppings.append('Tomato Sauce') # OOPS


AttributeError: 'PizzaSpecification' object has no attribute '__toppings'

In [38]:
# Python does name mangling when you prefix attributes with two underscores. 
# To find out what the new attribute names are, use the __dict__ attribute

pizza_spec.__dict__

{'_PizzaSpecification__dough_radius_in_inches': 8,
 '_PizzaSpecification__toppings': ['Olive Oil',
  'Garlic',
  'Sliced Roma Tomatoes',
  'Mozzarella']}

### Operations


In [41]:
# Suppose that for my pizza specification, I want to be able to add a topping 
# while the pizza is queued to be made. 
# I can define a new function that adds a topping, as follows

class PizzaException(Exception):
    pass

class PizzaSpecification:
    def __init__(
        self,
        dough_radius_in_inches: int,
        toppings: list[str]
    ):
        assert 6 <= dough_radius_in_inches <= 12, 'Dough must be between 6 and 12 inches'
        self.__dough_radius_in_inches = dough_radius_in_inches
        self.__toppings: list[str] = []
        for topping in toppings:
            self.add_topping(topping)

    def add_topping(self, topping: str):
        """
        Add a topping to the pizza
        All rules for pizza construction still apply.
        """
        if (
            is_sauce(topping) and 
            any(item for item in self.__toppings if is_sauce(item))
        ):
            raise PizzaException('Pizza may only have one sauce')

        if is_sauce(topping):
            self.__toppings.insert(0, topping)
        else:
            self.__toppings.append(topping)
